# 语义分析，或者语义生成
定义是：确定字符列或word sequences 意思的过程。可用于歧义消除的任务。

当分析一个输入的句子时，如果句子结构已经建立，那么可以进行该句的语义分析。

语义解释就是把意义映射到句子中。

语境解释（contextual interpretation）是将逻辑形式映射到knowledge representation。

语义分析的基本单位或基本单位称为meaning或sense。

几个处理senses 的工具：

    * ELIZA：它利用替换和模式匹配技术分析句子，并为给定的输入提供输出。
    * MARGIE：它可以用11个原语primitives 来表示所有的英语动词。
    * Script Applier Mechanism (SAM)：它可以翻译不同语言的句子，如英语、汉语、俄语、荷兰语和西班牙语。
    * 为了实现对文本数据的处理，要使用一个Python库或是TextBlob
    
语义分析可以用来查询数据库和检索信息。

    * 另一个Python库，Gensim，可以用来执行文件索引，主题建模，和相似性检索。
    * Polyglot 支持多种语言的应用的NLP工具。它提供40种不同语言的NER，165种for tokenization，165种for 语音分析……
    * MontyLingua用于对英语文本进行语义解释
    
# 句子可以用逻辑形式来表现。
命题逻辑 propositional logic 中的基本表达式或句子可用命题符号表示，比如P，Q，R等。
复数表达式可以用布尔运算符表示。

例如，用propositional logic 来表示句子：If it is raining, I'll wear a raincoat 

    • P: It is raining.
    • Q: I'll wear raincoat.
    • P->Q: If it is raining, I'll wear a raincoat.

## 一些操作符号：

In [1]:
import nltk
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


## Well-formed Formulas(WFF) 合语法的公式的构成：
使用propositional symbols 命题符号或 使用命题符号和上述boolean operators布尔操作符的组合

#### 将不同的逻辑表达式 归为不同的类：

In [5]:
import nltk
input_expr = nltk.sem.Expression.fromstring
input_expr('X | (Y -> Z)')

<OrExpression (X | (Y -> Z))>

In [6]:
input_expr('-(X & Y)')

<NegatedExpression -(X & Y)>

In [7]:
input_expr('X & Y')

<AndExpression (X & Y)>

In [8]:
input_expr('X <-> -- X')

<IffExpression (X <-> --X)>

使用***Valuation*** function to map **True** or **False** values to logical expressions

In [9]:
import nltk
value = nltk.Valuation([('X', True), ('Y', False), ('Z', True)])
value['Z']

True

In [10]:
domain = set()
v = nltk.Assignment(domain)
u = nltk.Model(domain, value)
print(u.evaluate('(X & Y)', v))
print(u.evaluate('-(X & Y)',v))
print(u.evaluate('(X & Z)', v))
print(u.evaluate('(X | Y)', v))

False
True
True
True


#### 涉及常数和谓词的一阶谓词逻辑：

In [11]:
import nltk
input_expr = nltk.sem.Expression.fromstring
expression = input_expr('run(marcus)', type_check=True)
expression.argument

<ConstantExpression marcus>

In [12]:
expression.argument.type

e

In [13]:
expression.function

<ConstantExpression run>

In [14]:
expression.function.type

<e,?>

In [15]:
sign = {'run': '<e, t>'}
expresion = input_expr('run(marcus)', signature = sign)
expression.function.type

<e,?>

#### signature用于映射相关的类型及non-logical constants，
to generate a query and retrieve data from the database:

In [17]:
import nltk
nltk.data.show_cfg('grammars/book_grammars/sql1.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
VP[SEM=(?v + ?np)] -> TV[SEM=?v] NP[SEM=?np]
VP[SEM=(?vp1 + ?c + ?vp2)] -> VP[SEM=?vp1] Conj[SEM=?c] VP[SEM=?vp2]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
NP[SEM=(?n + ?pp)]  -> N[SEM=?n] PP[SEM=?pp]
NP[SEM=?n]  -> N[SEM=?n]  | CardN[SEM=?n] 
CardN[SEM='1000'] -> '1,000,000' 
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
Conj[SEM='AND'] -> 'and'
N[SEM='City FROM city_table'] -> 'cities'
N[SEM='Population'] -> 'populations'
IV[SEM=''] -> 'are'
TV[SEM=''] -> 'have'
A -> 'located'
P[SEM=''] -> 'in'
P[SEM='>'] -> 'above'


In [19]:
from nltk import load_parser
test = load_parser('grammars/book_grammars/sql1.fcfg')
q = ' What cities are in Greece'  # 问：哪座城市在希腊
t = list(test.parse(q.split()))  # 通过解析出city,country 这些字段，通过sql语句在数据库中找
t

[Tree(S[SEM=(SELECT, City FROM city_table, WHERE, , , Country="greece")], [Tree(NP[SEM=(SELECT, City FROM city_table)], [Tree(Det[SEM='SELECT'], ['What']), Tree(N[SEM='City FROM city_table'], ['cities'])]), Tree(VP[SEM=(, , Country="greece")], [Tree(IV[SEM=''], ['are']), Tree(PP[SEM=(, Country="greece")], [Tree(P[SEM=''], ['in']), Tree(NP[SEM='Country="greece"'], ['Greece'])])])])]

In [22]:
t[0].label()

S[SEM=(SELECT, City FROM city_table, WHERE, , , Country="greece")]

In [24]:
ans = t[0].label()['SEM']
ans = [s for s in ans if s]
ans  

['SELECT', 'City FROM city_table', 'WHERE', 'Country="greece"']

In [25]:
q = ' '.join(ans)
q   # 得到sql query语句：

'SELECT City FROM city_table WHERE Country="greece"'

In [28]:
from nltk.sem import chat80
r = chat80.sql_query('corpora/city_database/city.db',q) # 参数q即sql语句，用该语句在city数据库中做sql查询
r

In [30]:
for p in r:    # 遍历结果
    print(p[0], end=' ') # Athens 雅典

athens 

# Introducing NER